<a href="https://colab.research.google.com/github/nachiketh25/capstone_project-ANZ-/blob/main/Cap_Fi_G4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import gspread
from google.auth import default
from google.colab import auth
import time
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('stock_data').sheet1

# Clear existing data and set up columns
worksheet.clear()
stocks = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
header = []
for stock in stocks:
    header.append(f"{stock} Time")
    header.append(f"{stock} Price")
worksheet.append_row(header)

def fetch_stock_data(stock_symbol):
    url = f"https://finance.yahoo.com/quote/{stock_symbol}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Upgrade-Insecure-Requests": "1"
    }
    time.sleep(2)
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except requests.exceptions.HTTPError as http_err:
        logging.warning(f"HTTP error occurred: {http_err}. Skipping stock symbol: {stock_symbol}")
        return None
    except Exception as err:
        logging.warning(f"Other error occurred: {err}. Skipping stock symbol: {stock_symbol}")
        return None

def extract_stock_data(soup, stock_symbol):
    if soup is None:
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S"), 'Error'

    logging.info(f"Extracting data for {stock_symbol}")

    try:
        price_element = soup.find('fin-streamer', {'data-symbol': stock_symbol, 'data-field': 'regularMarketPrice'})
        price = price_element.get_text(strip=True) if price_element else 'Price not found'
    except Exception as e:
        logging.error(f"Error extracting data for {stock_symbol}: {e}")
        price = 'Error'

    return datetime.now().strftime("%Y-%m-%d %H:%M:%S"), price

def update_sheet(row_data):
    worksheet.append_row(row_data)
    logging.info("Data saved to Google Sheet")

def main():
    while True:
        row_data = []
        for stock in stocks:
            soup = fetch_stock_data(stock)
            timestamp, price = extract_stock_data(soup, stock)
            row_data.append(timestamp)
            row_data.append(price)

        update_sheet(row_data)
        time.sleep(10)  # Sleep for 5 minutes before fetching again

if __name__ == '__main__':
    main()


MessageError: Error: credential propagation was unsuccessful